In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import datetime
import os
import sys
from get_unique_resources_data import get_unique_resources_data
from get_printable_resource_names import get_printable_resource_names
import matplotlib.pyplot as plt

In [2]:
case_names = [    
              "Thermal_Base",
              "2_Hr_BESS", 
              "2_Hr_BESS_Fuelx2",
              "4_Hr_BESS",
              "4_Hr_BESS_Fuelx2",
              "4_Hr_BESS_Fuelx3",
              "4_Hr_BESS_Fuelx4",
              "6_Hr_BESS",
              "6_Hr_BESS_Fuelx2",
              "8_Hr_BESS",
              "8_Hr_BESS_Fuelx2",
              "10_Hr_BESS",
              "10_Hr_BESS_Fuelx2",
              ]

current_dir = os.getcwd()
print(current_dir)

plots_path = os.path.join(current_dir, 'plots') + "/"
csv_path = os.path.join(current_dir, 'csv') + "/"
latex_path = os.path.join(current_dir, 'latex') + "/"
if not os.path.exists(plots_path):
    os.makedirs(plots_path)
if not os.path.exists(csv_path):
    os.makedirs(csv_path)
if not os.path.exists(latex_path):
    os.makedirs(latex_path)


cem_path = os.path.join(os.path.dirname(current_dir), 'GenX.jl', 'research_systems')
policies_path = os.path.join(os.path.dirname(current_dir), 'SPCM', 'research_systems')

date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

C:\Users\ks885\Documents\aa_research\Modeling\spcm_genx_experiment\figures


In [3]:
# get unique generator names
unique_resources, cases_resources_capacities = get_unique_resources_data(case_names, policies_path)


adding resource: NG 2-on-1 Combined Cycle (F-Frame) from case: Thermal_Base
adding resource: NG Combustion Turbine (F-Frame) from case: Thermal_Base
adding resource: Land-Based Wind - Class 1 - Technology 1 from case: Thermal_Base
adding resource: Utility PV - Class 1 from case: Thermal_Base
adding resource: Utility-Scale Battery Storage - 2Hr from case: 2_Hr_BESS
adding resource: Utility-Scale Battery Storage - 4Hr from case: 4_Hr_BESS
adding resource: Utility-Scale Battery Storage - 6Hr from case: 6_Hr_BESS
adding resource: Utility-Scale Battery Storage - 8Hr from case: 8_Hr_BESS
adding resource: Utility-Scale Battery Storage - 10Hr from case: 10_Hr_BESS


In [4]:
unique_resources

['NG 2-on-1 Combined Cycle (F-Frame)',
 'NG Combustion Turbine (F-Frame)',
 'Land-Based Wind - Class 1 - Technology 1',
 'Utility PV - Class 1',
 'Utility-Scale Battery Storage - 2Hr',
 'Utility-Scale Battery Storage - 4Hr',
 'Utility-Scale Battery Storage - 6Hr',
 'Utility-Scale Battery Storage - 8Hr',
 'Utility-Scale Battery Storage - 10Hr']

In [5]:
cases_resources_capacities_table = get_printable_resource_names(cases_resources_capacities)
# unique_resources_printable = get_printable_resource_names(unique_resources)

In [6]:
# Get the columns that start with 'BESS'
bess_columns = [col for col in cases_resources_capacities_table.columns if col.startswith('BESS')]

# Create a temporary vector to hold the maximum values of the BESS columns
bess_capacities = cases_resources_capacities_table[bess_columns].max(axis=1)

# Drop the original BESS columns
cases_resources_capacities_table_all = cases_resources_capacities_table.drop(columns=bess_columns)

# add the maximum BESS capacities to the table
cases_resources_capacities_table_all['BESS'] = bess_capacities



In [7]:
# Calculate the fraction of each resource for each case
fractions = cases_resources_capacities_table_all.copy()
resource_cols = [col for col in fractions.columns if col != 'Case Name']
fractions_total = fractions[resource_cols].sum(axis=1)
for col in resource_cols:
    fractions[col] = fractions[col] / fractions_total
fractions['Total'] = fractions[resource_cols].sum(axis=1)  # should be 1.0 for each row
# print to csv
fractions.to_csv(os.path.join(csv_path, 'table2b_capacity_in_percentage_float.csv'), index=False)
# convert numbers to percentages and remove decimal places and add a percent sign
fractions_percentage = fractions.copy()
fractions_percentage[resource_cols] = fractions_percentage[resource_cols].applymap(lambda x: f"{int(round(x * 100, 0))}%")
# print percentage to csv
fractions_percentage.to_csv(os.path.join(csv_path, 'table2b_capacity_in_percentage_str.csv'), index=False)


In [8]:
# Round the values in the table to zero decimal places and convert to integers
cases_resources_capacities_table_rounded = cases_resources_capacities_table_all.copy()
columns_to_round = [col for col in cases_resources_capacities_table_rounded.columns if col != 'Case Name']
cases_resources_capacities_table_rounded[columns_to_round] = cases_resources_capacities_table_rounded[columns_to_round].round(0).astype(int)

In [9]:
cases_resources_capacities_table_all

,Case Name,NG CC,NG CT,Wind,Solar,BESS
0,Thermal_Base,46.274023,9.457160,32.791061,17.972935,0.000000
1,2_Hr_BESS,34.646970,12.954406,49.965193,25.930402,13.039450
2,2_Hr_BESS_Fuelx2,27.226989,12.359484,74.051006,45.098958,45.216721
3,4_Hr_BESS,33.500065,11.691107,53.880599,23.917770,16.257362
4,4_Hr_BESS_Fuelx2,26.335443,12.038484,72.559042,46.470854,32.857363
5,4_Hr_BESS_Fuelx3,24.713901,10.732273,77.868475,60.816319,41.107589
6,4_Hr_BESS_Fuelx4,24.749554,8.188353,79.198094,68.234310,50.674644
7,6_Hr_BESS,33.852765,10.804578,54.948394,21.498364,13.085191
8,6_Hr_BESS_Fuelx2,26.102852,12.107401,70.503699,48.438166,23.528685
9,8_Hr_BESS,34.412069,10.078836,55.254771,20.432080,10.572517


In [10]:
# save the table to a csv file in tables_path, and latex format in latex_path
cases_resources_capacities_table_rounded.to_csv(os.path.join(csv_path, 'table2_capacity_in_gw.csv'), index=False)


In [11]:
latex_file = os.path.join(latex_path, 'table2_capacity_in_gw.tex')

with open(latex_file, 'w') as f:
    f.write('\\begin{table}[htp]\n')
    f.write('\\centering\n')
    f.write('\\begin{tabular}{lcccccc}\n')
    f.write('\\hline\n')
    f.write('\\textbf{Case} & \\textbf{NG CC} & \\textbf{NG CT} & \\textbf{Wind} & \\textbf{Solar} & \\textbf{Battery} & \\textbf{Total} \\\\ \\hline\n')
    for idx, row in cases_resources_capacities_table_rounded.iterrows():
        case = row['Case Name'].replace('_', '\\_')
        ng_cc = row['NG CC']
        ng_ct = row['NG CT']
        wind = row['Wind']
        solar = row['Solar']
        battery = row['BESS']
        total = ng_cc + ng_ct + wind + solar + battery
        case_print = case.replace('_', ' ')
        f.write(f'{case_print} & {ng_cc} & {ng_ct} & {wind} & {solar} & {battery} & {total} \\\\\n')
    f.write('\\hline\n')
    f.write('\\end{tabular}\n')
    f.write('\\caption{Capacity in GW of each resource in the constructed cases}\n')
    f.write('\\label{table:all_cases}\n')
    f.write('\\end{table}\n')

In [12]:
latex_file = os.path.join(latex_path, 'table2b_capacity_in_percentages.tex')

with open(latex_file, 'w') as f:
    f.write('\\begin{table}[htp]\n')
    f.write('\\centering\n')
    f.write('\\begin{tabular}{lcccccc}\n')
    f.write('\\hline\n')
    f.write('\\textbf{Case} & \\textbf{NG CC} & \\textbf{NG CT} & \\textbf{Wind} & \\textbf{Solar} & \\textbf{Battery} & \\textbf{Total} \\\\ \\hline\n')
    for idx, row in cases_resources_capacities_table_all.iterrows():
        case = row['Case Name'].replace('_', '\\_')
        ng_cc = row['NG CC']
        ng_ct = row['NG CT']
        wind = row['Wind']
        solar = row['Solar']
        battery = row['BESS']
        total = ng_cc + ng_ct + wind + solar + battery
        ng_cc_perc = f"{int(round(ng_cc * 100 / total, 0))}\%"
        ng_ct_perc = f"{int(round(ng_ct * 100 / total, 0))}\%"
        wind_perc = f"{int(round(wind * 100 / total, 0))}\%"
        solar_perc = f"{int(round(solar * 100 / total, 0))}\%"
        battery_perc = f"{int(round(battery * 100 / total, 0))}\%"

        case_print = case.replace('_', ' ')
        f.write(f'{case_print} & {ng_cc_perc} & {ng_ct_perc} & {wind_perc} & {solar_perc} & {battery_perc} & {total} \\\\\n')
    f.write('\\hline\n')
    f.write('\\end{tabular}\n')
    f.write('\\caption{Capacity fraction of each resource and total system capacity in GW}\n')
    f.write('\\label{table:all_cases}\n')
    f.write('\\end{table}\n')

In [13]:
# cases_resources_capacities_table_rounded.to_latex(os.path.join(latex_path, 'table2_capacity_in_gw.tex'), index=False, escape=False)